# DeployMe - Migration Based Deployment

For this session's purpose, creating a manual version of a Migration State deployment. 

  

\* DeployMe\_MigrationScript.ps1 is a fairly simple PowerShell script that will go through the directory and run the scripts in the specified order and log the entry to a MigationLog table.

## Reset the Database

Pull down the .NET interactive packages for connecting to SQL Server before starting.

NOTE. You can also specify the specific version of the code you want to use. If so, it could look something like this:

```
#r "nuget: Microsoft.DotNet.Interactive.SqlServer, 1.0.0-beta.22504.6"

```

Then drop and recreate the database so we have an empty shell of database

In [ ]:
#r "nuget:Microsoft.DotNet.Interactive.SqlServer,1.0.0-beta.22504.6"

In [ ]:
#!connect mssql --kernel-name mymasterdb "Persist Security Info=False; Integrated Security=true; Initial Catalog=master; Server=localhost\sql2019"

In [ ]:
#!sql-mymasterdb

DROP DATABASE DeployMe_Migration

CREATE DATABASE DeployMe_Migration


## Run the initial PowerShell script to run all the files

In [ ]:
. "C:\GitRepo\DBDeployments\DeployMe\DeployMe_Migration\DeployMe_MigrationScript.ps1"

Confirm that objects exists

In [ ]:
#!connect mssql --kernel-name DeployMeM "Persist Security Info=False; Integrated Security=true; Initial Catalog=DeployMe_Migration; Server=localhost\sql2019"

In [ ]:
#!sql-DeployMeM

SELECT TOP 10 name, schema_id FROM sys.tables

SELECT TOP 10 * FROM Deploy.MigrationLog
ORDER BY MigrationLogID DESC 


## Add Data File

There is a data script in "C:\\GitRepo\\DBDeployments\\DeployMe\\Demo Files\\Lookup10.Data.sql" that we want to add to a Data folder in the Migration script directory. The PowerShell script should be looking for it and adding it automatically. 

Steps are:

1. Select from the table to prove there are no records
2. Copy the file into a Data directory
3. Run the PowerShell script
4. Run the Select again

In [ ]:
#!sql-DeployMeM

SELECT * FROM dbo.Lookup10

In [ ]:
New-Item -Path 'C:\GitRepo\DBDeployments\DeployMe\DeployMe_Migration\Data' -ItemType Directory
Copy-item -Path "C:\GitRepo\DBDeployments\DeployMe\Demo Files\Lookup10.Data.sql" -Destination "C:\GitRepo\DBDeployments\DeployMe\DeployMe_Migration\Data\Lookup10.Data.sql" -Force

In [ ]:
. "C:\GitRepo\DBDeployments\DeployMe\DeployMe_Migration\DeployMe_MigrationScript.ps1"

In [ ]:
#!sql-DeployMeM

SELECT * FROM dbo.Lookup10

## Schema Change

How would we handle schema changes?

  

Some of these changes could be done by script but for now, I will manually walk through the steps: 

1. Add script for new schema, DemoSetup
2. Show the stored proc script DummyTableAndProcCreation to move to the new schema.
1. Reminder: Rename in Git\\Source Control to keep the file history.
4. Run the PowerShell script (optional)

In [ ]:
. "C:\GitRepo\DBDeployments\DeployMe\DeployMe_Migration\DeployMe_MigrationScript.ps1"

In [ ]:
#!sql-DeployMeM

SELECT schema_name(schema_id) FROM sys.procedures 
WHERE Name = 'DummyTableAndProcCreation'